In [1]:
import os
import torch
import numpy as np
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from utils import *
from model import *

seed=816
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [2]:
# ---------------------
# parameters
# ---------------------
lr = 2e-3
epochs = 100
batch_size = 1
pos_weights = 1
path = 'D:/graph-conflation-data/'

# ---------------------
# load data
# ---------------------
print('Load Datasets...')
files = os.listdir(path+'/graphs/osm/')
train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
train_files, val_files = train_test_split(train_files, test_size=0.2, random_state=42)

# make datasets
train_data = GraphDataset(path, train_files)
val_data = GraphDataset(path, val_files)
test_data = GraphDataset(path, test_files)

# data loader
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False) 

Load Datasets...


In [3]:
graph_acc_intersection = []
graph_acc_union = []
for file in tqdm(test_files):
    # paths
    osw_path = os.path.join(path, 'graphs/osw/', file)
    osm_path = os.path.join(path, 'graphs/osm/', file)
    sdot_path = os.path.join(path, 'graphs/sdot/', file)
    
    # graphs & edges
    graph_osw = nx.to_numpy_array(nx.read_graph6(osw_path))
    graph_osm = nx.to_numpy_array(nx.read_graph6(osm_path))
    graph_sdot = nx.to_numpy_array(nx.read_graph6(sdot_path))
    union = 1*np.logical_or(graph_osm,graph_sdot)
    intersection = 1*np.logical_and(graph_osm,graph_sdot)
    graph_acc_union.append(np.mean(1.0*( union == graph_osw)))    
    graph_acc_intersection.append(np.mean(1.0*( intersection == graph_osw)))

100%|█████████████████████████████████████████████████████████████████████████████| 1773/1773 [00:02<00:00, 775.39it/s]


In [4]:
np.mean(graph_acc_intersection), np.mean(graph_acc_union)

(0.8965330700741877, 0.8446568998296642)

In [5]:
edge_acc_intersection = []
edge_acc_union = []
for file in tqdm(test_files):
    # paths
    osw_path = os.path.join(path, 'graphs/osw/', file)
    osm_path = os.path.join(path, 'graphs/osm/', file)
    sdot_path = os.path.join(path, 'graphs/sdot/', file)
    
    # graphs & edges
    graph_osw = nx.to_numpy_array(nx.read_graph6(osw_path))
    graph_osm = nx.to_numpy_array(nx.read_graph6(osm_path))
    graph_sdot = nx.to_numpy_array(nx.read_graph6(sdot_path))    
    graph_osm_edges = np.array(nx.read_graph6(osm_path).edges()).T
    graph_sdot_edges = np.array(nx.read_graph6(sdot_path).edges()).T
    
    # extract conflicted edges
    conflicted_edges = np.concatenate(
        [graph_osm_edges.T[~(graph_osm_edges.T[:, None] == graph_sdot_edges.T).all(-1).any(-1)],
         graph_sdot_edges.T[~(graph_sdot_edges.T[:, None] == graph_osm_edges.T).all(-1).any(-1)]],
        axis=0
    )
    conflicted_edges = np.repeat(conflicted_edges, 2,axis=0)
    conflicted_edges[::2,[0,1]] = conflicted_edges[::2,[1,0]]
    conflicted_edges = list(zip(*conflicted_edges.T))  
    
    
    union = 1*np.logical_or(graph_osm,graph_sdot)
    intersection = 1*np.logical_and(graph_osm,graph_sdot)
    
    edge_acc_intersection.append(np.mean([1.0*(intersection[c]==graph_osw[c]) for c in conflicted_edges]))
    edge_acc_union.append(np.mean([1.0*(union[c]==graph_osw[c]) for c in conflicted_edges]))

 22%|████████████████▊                                                             | 382/1773 [00:00<00:01, 736.07it/s]C:\Users\binha\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\binha\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|█████████████████████████████████████████████████████████████████████████████| 1773/1773 [00:02<00:00, 664.31it/s]


In [6]:
## 5
np.mean(np.array(edge_acc_intersection)[~np.isnan(edge_acc_intersection)])

0.8349301370819641